# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_df', con=engine)


In [4]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [12]:
df.related.unique()

array([1, 0, 2])

In [5]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [6]:
df.columns[5:]

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [3]:
X = df['message']
y = df[df.columns[5:]]
dum = pd.get_dummies(df[['related','genre']])
y = pd.concat([y, dum], axis=1)

In [11]:
y.columns

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report',
       'related', 'genre_direct', 'genre_news', 'genre_social'],
      dtype='object')

In [13]:
y.related.unique()

array([1, 0, 2])

In [4]:
y.related.replace(2,1,inplace=True)

In [5]:
y.related.unique()

array([1, 0])

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    
    # replace url
    url_regex=r"""((?:(?:https|ftp|http)?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|org|uk)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|uk|ac)\b/?(?!@)))"""
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, "urlconverted")
    
    # remove punctuation 
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    newtokens = []
    for tok in tokens:
        newtok = lemmatizer.lemmatize(tok).lower().strip()
        newtokens.append(newtok)
    return newtokens
   

In [33]:
tokenize(df['message'][0])

['weather',
 'update',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [5]:
CountVectorizer(tokenizer=tokenize(df['message'][1]))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'],
        vocabulary=None)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
clff = MultiOutputClassifier(RandomForestClassifier())
pipeline =  Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', clff)
    ])

In [12]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [14]:
# fit pipeline
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [62]:
y_pred = pipeline.predict(X_test)

In [22]:
y_pred.shape

(6554, 39)

In [24]:
y_test.shape

(6554, 39)

In [25]:
y_test.columns

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report',
       'related', 'genre_direct', 'genre_news', 'genre_social'],
      dtype='object')

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
 results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])

In [15]:
def model_result(y_test,y_pred):
    final = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    for i,cat in enumerate(y_test.columns):
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,i], average='weighted')
        final.loc[i,'Category']=cat
        final.loc[i,'f_score']=f_score
        final.loc[i,'precision']=precision
        final.loc[i,'recall']=recall
    
    print('Aggregate f_score:', final['f_score'].mean())
    print('Aggregate precision:', final['precision'].mean())
    print('Aggregate recall:', final['recall'].mean())
    return final

In [66]:
model_result(y_test,y_pred)

Aggregate f_score: 0.92918338496
Aggregate precision: 0.931554287041
Aggregate recall: 0.942270525731


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,f_score,precision,recall
0,request,0.858808,0.872214,0.877632
1,offer,0.994206,0.992385,0.996033
2,aid_related,0.718052,0.732395,0.728715
3,medical_help,0.888322,0.896164,0.918523
4,medical_products,0.929229,0.93286,0.949496
5,search_and_rescue,0.957011,0.95625,0.969179
6,security,0.972772,0.964013,0.981691
7,military,0.95394,0.95582,0.96689
8,child_alone,1,1,1
9,water,0.9314,0.940227,0.945835


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb868ba2f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [18]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = cv.predict(X_test)
model_result(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Aggregate f_score: 0.930173554554
Aggregate precision: 0.933740989304
Aggregate recall: 0.944066258226


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,f_score,precision,recall
0,request,0.864202,0.879101,0.88404
1,offer,0.994282,0.992386,0.996186
2,aid_related,0.742793,0.744049,0.746262
3,medical_help,0.891624,0.896584,0.922795
4,medical_products,0.928595,0.943707,0.951022
5,search_and_rescue,0.958841,0.959257,0.972231
6,security,0.97027,0.96072,0.980012
7,military,0.945821,0.948268,0.962313
8,child_alone,1,1,1
9,water,0.934232,0.944534,0.948581


In [20]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [22]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# testing a pure decision tree classifier
clff = MultiOutputClassifier(AdaBoostClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', clff)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)
model_result(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


Aggregate f_score: 0.941288114051
Aggregate precision: 0.940633704598
Aggregate recall: 0.947618600502


,Category,f_score,precision,recall
0,request,0.889083,0.889374,0.896094
1,offer,0.994738,0.993599,0.99588
2,aid_related,0.749533,0.755709,0.755417
3,medical_help,0.911007,0.909474,0.925236
4,medical_products,0.946962,0.946146,0.954684
5,search_and_rescue,0.966899,0.966845,0.973604
6,security,0.969961,0.967797,0.978944
7,military,0.968412,0.968474,0.972841
8,child_alone,1,1,1
9,water,0.962892,0.962243,0.964602


Apparently, the above model is the best model of all models we have built.

### 9. Export your model as a pickle file

In [23]:
pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.